In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from concurrent.futures import ThreadPoolExecutor

# Initialize a browser with options
def initialize_browser():
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run in headless mode
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument(
        'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    )
    return webdriver.Chrome(service=service, options=options)

# Handle CAPTCHA (manual)
def handle_captcha(driver):
    try:
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//iframe[contains(@src, 'captcha')]"))
        )
        time.sleep(30)  # Pause for manual solving
    except Exception:
        pass

# Clean the content
def clean_content(text):
    return text.replace("Click to expand...", "").strip()

# Scrape content from individual post pages
def scrape_bb_wrapper_page(driver, url, post_title, results):
    driver.get(url)
    time.sleep(random.uniform(5, 10))
    handle_captcha(driver)
    
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    message_containers = soup.find_all('div', class_='js-messageContent')
    
    for container in message_containers:
        try:
            likes = "0"
            user_extras = container.find_previous('div', class_='message-userExtras')
            if user_extras:
                likes_element = user_extras.select_one('dl:nth-of-type(3) dd')
                likes = likes_element.text.strip() if likes_element else "0"
            
            bb_wrapper = container.find('div', class_='bbWrapper')
            if bb_wrapper:
                text = bb_wrapper.get_text(separator=" ", strip=True)
                cleaned_text = clean_content(text)
                
                username_span = container.find('span', class_=lambda x: x and 'username' in x)
                if not username_span:
                    username_span = container.find_previous('span', class_=lambda x: x and 'username' in x)
                username = username_span.get_text(strip=True) if username_span else "Unknown"
                
                time_tag = container.find_previous('time', class_='u-dt')
                date = time_tag['data-date'] if time_tag else "Unknown Date"
                
                results.append({
                    "PostTitle": post_title,
                    "Username": username,
                    "Date": date,
                    "Likes": likes,
                    "Content": cleaned_text
                })
        except Exception:
            pass

# Scrape a single page of the forum
def scrape_forum_page(driver, url, page_number, results):
    page_url = f"{url}page-{page_number}" if page_number > 1 else url
    driver.get(page_url)
    time.sleep(random.uniform(5, 10))
    handle_captcha(driver)
    
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    thread_rows = soup.find_all('div', class_='structItem')
    
    for thread in thread_rows:
        try:
            title_element = thread.find('div', class_='structItem-title')
            if title_element:
                anchor = title_element.find('a', href=True)
                link = anchor['href'] if anchor else "#"
                
                if "sticky-threads" in link:
                    continue  # Skip sticky threads
                
                post_title = anchor.text.strip() if anchor else "Unknown Title"
                full_link = f"https://www.blackhatworld.com{link}" if link.startswith('/') else link
                print(full_link)  # Debugging output
                scrape_bb_wrapper_page(driver, full_link, post_title, results)
        except Exception:
            pass

# Parallel processing for forum scraping
def scrape_black_hat_seo_forum_parallel(url, start_page, end_page):
    results = []
    max_threads = 10  # Number of threads for parallel processing

    def scrape_page(page_number):
        driver = initialize_browser()
        try:
            scrape_forum_page(driver, url, page_number, results)
        finally:
            driver.quit()

    with ThreadPoolExecutor(max_threads) as executor:
        executor.map(scrape_page, range(start_page, end_page + 1))
    
    return results

# Main function to scrape the forum
def main():
    forum_url = "https://www.blackhatworld.com/forums/black-hat-seo.28/"
    start_page = 2301
    end_page = 2700  # Define the range of pages to scrape
    
    results = scrape_black_hat_seo_forum_parallel(forum_url, start_page, end_page)
    
    if results:
        df = pd.DataFrame(results, columns=["PostTitle", "Username", "Date", "Likes", "Content"])
        output_file = "C:\\Users\\USER\\Downloads\\ForumContent_2301_to_2700.xlsx"
        df.to_excel(output_file, index=False)
        print(f"Data successfully fetched and stored in {output_file}.")

# Run the scraper
if __name__ == "__main__":
    main()


https://www.blackhatworld.com/seo/a-practice-that-is-still-effective.848120/
https://www.blackhatworld.com/seo/hostug-closed-its-business-who-is-the-service-provider-now.848917/
https://www.blackhatworld.com/seo/how-long-before-linking-your-money-site-to-your-pbn.847425/
https://www.blackhatworld.com/seo/ranking-but-little-traffic.849409/
https://www.blackhatworld.com/seo/how-is-it-possible.849538/
https://www.blackhatworld.com/seo/do-you-think-google-spies-on-seos-using-chrome-or-no.847761/
https://www.blackhatworld.com/seo/advice-on-a-keyword-needed.848725/
https://www.blackhatworld.com/seo/im-looking-to-create-a-seed-list-of-10-000-emails.848266/
https://www.blackhatworld.com/seo/set-up-a-new-ip-for-each-website-meaning.848807/
https://www.blackhatworld.com/seo/can-google-penalise-for-exist-confirm-alert.849917/
https://www.blackhatworld.com/seo/what-affiliate-programs-convert-best-with-facebook-ads.848095/
https://www.blackhatworld.com/seo/why-even-after-82-clicks-i-am-getting-15-f

https://www.blackhatworld.com/seo/confused-about-spamming-money-site.847290/
https://www.blackhatworld.com/seo/worried-about-buying-expired-domains-for-pbn.847256/
https://www.blackhatworld.com/seo/guide-how-to-spamcheck-expired-domains-the-easy-way.847919/
https://www.blackhatworld.com/seo/casino-site-hacked-my-business-website.847878/
https://www.blackhatworld.com/seo/looking-for-spanish-writers-mexican-spanish.847967/
https://www.blackhatworld.com/seo/does-free-whoisguard-on-namecheap-stay-private-after-you-transfer-the-domain-to-a-host.849899/
https://www.blackhatworld.com/seo/imdb.843996/
https://www.blackhatworld.com/seo/link-trades-add-your-adult-sites.847335/
https://www.blackhatworld.com/seo/what-captcha-solving-service-to-use.410081/
https://www.blackhatworld.com/seo/domain-extensions-and-seo.848935/
https://www.blackhatworld.com/seo/site-rankings-dissappeared.842459/
https://www.blackhatworld.com/seo/looking-for-high-traffic-keywords-with-low-competition.847625/
https://www.

https://www.blackhatworld.com/seo/how-many-articles-do-you-spin-per-project-in-gsa-ser.848012/
https://www.blackhatworld.com/seo/your-recommendations-for-bhw-link-building-services.847428/
https://www.blackhatworld.com/seo/need-help-regarding-crackrevenue-type-of-promotions.849194/
https://www.blackhatworld.com/seo/tumblr-sape-gsa-method.802664/
https://www.blackhatworld.com/seo/ask-fake-refferer-plugin-wordpress.849455/
https://www.blackhatworld.com/seo/alternative-to-atomic-mail-verifier-to-do-bounce-check-before-bulk-mailing.795086/
https://www.blackhatworld.com/seo/question-about-proxies-and-scraping.849014/
https://www.blackhatworld.com/seo/what-is-the-deal-with-html5-sites.847258/
https://www.blackhatworld.com/seo/how-to-start-a-movie-steaming-web-site.846236/
https://www.blackhatworld.com/seo/301-from-2-0s.847754/
https://www.blackhatworld.com/seo/want-to-trade-1-legit-yelp-review-for-another-thats-it.726755/
https://www.blackhatworld.com/seo/any-service-to-solve-the-confounding

https://www.blackhatworld.com/seo/how-do-you-manage-all-different-your-webmaster-and-hosting-accounts.846028/
https://www.blackhatworld.com/seo/dont-like-someone-or-would-you-like-to-test-your-own-email-inbox-on-spam.761753/
https://www.blackhatworld.com/seo/best-of-the-world.845724/
https://www.blackhatworld.com/seo/free-seo-tool-for-newbies.846196/
https://www.blackhatworld.com/seo/any-risk-changing-pbn-anchor-url.845821/
https://www.blackhatworld.com/seo/wta-how-to-change-the-blog-post-date-shown-in-the-search-result.846689/
https://www.blackhatworld.com/seo/bad-to-change-titles-and-url-once-published.847008/
https://www.blackhatworld.com/seo/email-marketing.341350/
https://www.blackhatworld.com/seo/best-indexing-service-case-study-or-how-to-get-links-indexed-for-free.791046/
https://www.blackhatworld.com/seo/should-i-upload-videos-to-sites-other-than-youtube.846168/
https://www.blackhatworld.com/seo/where-can-i-find-free-database-dumps.312538/
https://www.blackhatworld.com/seo/purc

https://www.blackhatworld.com/seo/adsense-new-mobile-ads.845854/
https://www.blackhatworld.com/seo/do-you-think-i-will-get-a-refund-from-godaddy.845996/
https://www.blackhatworld.com/seo/how-do-you-guys-pick-which-domain-to-use-for-pbn-and-money-site.845965/
https://www.blackhatworld.com/seo/estimate-how-much-it-would-cost-to-rank-for-best-coffee-maker-or-best-hair-dryer.846489/
https://www.blackhatworld.com/seo/ranking-for-uk-google.845523/
https://www.blackhatworld.com/seo/best-cpm-ad-networks.845547/
https://www.blackhatworld.com/seo/getting-around-craigslists-179-character-search-limit.844349/
https://www.blackhatworld.com/seo/how-long-does-it-usually-take-to-rank-a-fresh-domain.845754/
https://www.blackhatworld.com/seo/adland-media-revenuehits-scam-or-legit.845655/
https://www.blackhatworld.com/seo/how-to-index-link-fast.734578/
https://www.blackhatworld.com/seo/advice.845390/
https://www.blackhatworld.com/seo/fiverr-reviews.770184/
https://www.blackhatworld.com/seo/how-can-i-harm

...

https://www.blackhatworld.com/seo/help-with-scraping-project-maybe-you-can-point-me.795950/
https://www.blackhatworld.com/seo/how-to-find-out-what-efforts-competitors-are-using-in-terms-of-local-seo-citations.796195/
https://www.blackhatworld.com/seo/need-help-with-email-send-method-technique.796298/
https://www.blackhatworld.com/seo/question-re-expired-domain-good-pa-da-cf-tf-but.795242/
https://www.blackhatworld.com/seo/supreme-domain-301-pbn-or-sell.796336/
https://www.blackhatworld.com/seo/traffic-bot.759767/
https://www.blackhatworld.com/seo/what-should-my-tier-1-consist-of.796387/
https://www.blackhatworld.com/seo/question-about-web-2-0-buffers-and-long-tail-lsi-keywords.796016/
https://www.blackhatworld.com/seo/best-links-to-buy-to-rank-a-twitter-profile-in-serps.796702/
https://www.blackhatworld.com/seo/experiments-with-sandbox-and-strange-results-help.794698/
https://www.blackhatworld.com/seo/registering-a-domain-at-namecheap-help.795744/
https://www.blackhatworld.com/seo/any-

https://www.blackhatworld.com/seo/one-of-my-websites-show-this-error-how-can-i-solve-this-issue.794426/
https://www.blackhatworld.com/seo/would-you-be-interested-in-landing-page-cloak-service.795047/
https://www.blackhatworld.com/seo/does-anyone-know-where-to-find-pics-of-sexy-women-for-im.775542/
https://www.blackhatworld.com/seo/linkbuilding-on-zendesk-qatarairways-website.588603/
https://www.blackhatworld.com/seo/what-are-the-tools-needed-for-mass-emailing.341951/
https://www.blackhatworld.com/seo/does-anybody-here-run-a-seotool-on-amazon-cloud-aws-ec2.795210/
https://www.blackhatworld.com/seo/i-have-a-ton-of-social-traffic-where-to-send-to-exchange-for-different-traffic.795278/
https://www.blackhatworld.com/seo/looking-for-a-keyword-research-tool-to-save-time-suggestions-much-appreciated.794526/
https://www.blackhatworld.com/seo/cpagrip-or-cpalead.794660/
https://www.blackhatworld.com/seo/frames-a-pbn-question.795727/
https://www.blackhatworld.com/seo/churn-burn-sites-can-i-get-som

https://www.blackhatworld.com/seo/negative-seo.792668/
https://www.blackhatworld.com/seo/need-someone-help.794278/
https://www.blackhatworld.com/seo/how-to-have-multiple-accounts-of-vps-server.793792/
https://www.blackhatworld.com/seo/please-your-2-minute-can-make-my-life.794964/
https://www.blackhatworld.com/seo/mass-planner-follow-settings.793874/
https://www.blackhatworld.com/seo/need-amazon-aws-accounts-with-ses-production-access.795113/
https://www.blackhatworld.com/seo/custom-gsa-list.794462/
https://www.blackhatworld.com/seo/how-rank-without-content.793914/
https://www.blackhatworld.com/seo/social-locker-for-html-css-websites.794798/
https://www.blackhatworld.com/seo/google-and-copy-paste.785050/
https://www.blackhatworld.com/seo/buying-traffic-at-30-cents-per-click-for-blackhat-offers.794316/
https://www.blackhatworld.com/seo/301-this-domain-or-use-as-pbn.793424/
https://www.blackhatworld.com/seo/is-anyone-part-of-jerry-wests-private-seo-revolution-group.794991/
https://www.bla

https://www.blackhatworld.com/seo/youtube-strategy-for-long-videos.793711/
https://www.blackhatworld.com/seo/adsense-click-fraud-plugin-check-this.793213/
https://www.blackhatworld.com/seo/does-having-related-posts-count-as-internal-linking.793296/
https://www.blackhatworld.com/seo/strange-link-building.792822/
https://www.blackhatworld.com/seo/optimizepress-2-vs-optin-revolution-pro.792349/
https://www.blackhatworld.com/seo/someone-tried-to-hack-my-site.792740/
https://www.blackhatworld.com/seo/delete-account.793589/
https://www.blackhatworld.com/seo/how-can-i-knockout-competition.792882/
https://www.blackhatworld.com/seo/adsense-code.792558/
https://www.blackhatworld.com/seo/emails-with-forwarding.793850/
https://www.blackhatworld.com/seo/is-country-blocking-may-cause-negative-effect-on-seo.793119/
https://www.blackhatworld.com/seo/is-there-any-tools-to-check-over-optimized-onpage.793281/
https://www.blackhatworld.com/seo/what-is-meant-by-backlinks.792807/
https://www.blackhatworld.c

https://www.blackhatworld.com/seo/how-reverse-image-search-works.792844/
https://www.blackhatworld.com/seo/pay-on-result-service-low-comp-keyword-in-13th-position.790190/
https://www.blackhatworld.com/seo/how-to-clean-send-to-100k-cold-email-list.792179/
https://www.blackhatworld.com/seo/micro-niche-site.753819/
https://www.blackhatworld.com/seo/how-to-get-indexed-in-google.791793/
https://www.blackhatworld.com/seo/is-this-website-doing-seo.791913/
https://www.blackhatworld.com/seo/best-pbn-finder-tool.791085/
https://www.blackhatworld.com/seo/any-redbutton-users-here.652944/
https://www.blackhatworld.com/seo/want-to-buy-uk-traffic-high.792453/
https://www.blackhatworld.com/seo/i-just-created-a-blogging-platform.792629/
https://www.blackhatworld.com/seo/burner-phone-numbers-looking-to-find.792171/
https://www.blackhatworld.com/seo/i-would-like-to-make-fake-game-hack-generators.791528/
https://www.blackhatworld.com/seo/cheapest-way-to-drive-10k-visitors-to-a-site-link.792969/
https://ww